## Chat Grupal de Agentes - Caso de Uso de un Canal de Noticias (Creación de Noticias - Escritor y Revisor)

![Flujo de Trabajo del Agente](https://github.com/kuljotSB/semantic-kernel/blob/main/Assets/AgentChat.jpg?raw=true)


#### Instalando SDK y Bibliotecas

In [ ]:
%pip install semantic-kernel==1.28.1, azure-identity, python-dotenv, azure-ai-projects==1.0.0b7

#### Creando el Cliente de Azure AI Project

In [1]:
# --- Importaciones Necesarias ---
from azure.identity import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MessageTextContent
from dotenv import load_dotenv
import asyncio
from semantic_kernel import Kernel
# Importa la clase principal para gestionar el chat grupal.
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
# Importa las clases de 'estrategia' que controlarán el flujo de la conversación.
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.functions import KernelFunctionFromPrompt

# Se carga el archivo .env.
load_dotenv()

# Se obtiene el nombre del modelo a utilizar.
model = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL")

# Se crea el cliente para conectar con el Servicio de Agentes de Azure AI.
project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(),
                                          conn_str=os.getenv("AI_PROJECT_CONNECTION_STRING")
)

#### Nombrando a nuestros agentes "Escritor" y "Revisor"

In [2]:
# Se definen los nombres de los agentes como constantes para evitar errores de escritura.
REVIEWER_NAME = "Revisor"
WRITER_NAME = "Escritor"

#### Prompt del Sistema para el Agente "Escritor"

In [3]:
# Estas son las instrucciones que definen la personalidad y el único propósito del agente Escritor.
writer_agent_prompt = f"""Tu única responsabilidad es reescribir contenido de acuerdo a las sugerencias de la revisión.
- Aplica siempre todas las indicaciones de la revisión.
- Revisa siempre el contenido en su totalidad y sin dar explicaciones.
- Nunca te dirijas al usuario."""

#### Creando el Agente "Escritor"

In [4]:
# Se crea la definición del agente Escritor en el servicio de Azure AI.
writer_agent_definition = await project_client.agents.create_agent(
        model=model,
        name=WRITER_NAME,
        instructions=writer_agent_prompt, # Se le asignan sus instrucciones específicas.
    )

# Se crea el objeto 'control remoto' de Semantic Kernel para el agente Escritor.
agent_writer = AzureAIAgent(
        client=project_client,
        definition=writer_agent_definition,
    )

#### Prompt del Sistema para el Agente "Revisor"

In [5]:
# Instrucciones detalladas que definen el comportamiento del agente Revisor.
reviewer_agent_prompt = f"""Eres un agente revisor. Tu tarea es evaluar un artículo corto (~100 palabras) escrito por otro agente.
Revísalo basándote en los siguientes criterios:

1. Claridad – ¿Es el mensaje claro y fácil de seguir?
2. Precisión – ¿Parece el contenido factualmente correcto?
3. Interés – ¿Es interesante y apropiado para una audiencia general?
4. Lenguaje – ¿Hay problemas de gramática o estilo?

Proporciona:

1. Una evaluación breve (2–3 líneas).
2. Una puntuación entre 1–10 que refleje la calidad general del artículo.
3. Si la puntuación es 8 o inferior, proporciona de 1 a 3 sugerencias para mejorar el artículo.
4. Si la puntuación es superior a 8, simplemente responde: "El artículo está listo para publicarse."

No pidas más refinamientos a menos que la puntuación sea 8 o inferior."""


#### Creando el Agente "Revisor"

In [6]:
# Se crea la definición del agente Revisor en el servicio de Azure AI.
reviewer_agent_definition = await project_client.agents.create_agent(
        model=model,
        name=REVIEWER_NAME,
        instructions=reviewer_agent_prompt, # Se le asignan sus reglas de revisión.
    )

# Se crea el objeto 'control remoto' de Semantic Kernel para el agente Revisor.
agent_reviewer = AzureAIAgent(
        client=project_client,
        definition=reviewer_agent_definition,
    )

#### Creando el Kernel

In [8]:
# Se crea una instancia del Kernel. Este Kernel se usará para ejecutar las 'estrategias' del chat.
kernel = Kernel()

load_dotenv()
api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

service_id = "service1"
# Se añade un servicio de IA al Kernel. Las estrategias lo usarán para tomar decisiones.
kernel.add_service(
    AzureChatCompletion(service_id=service_id,
                        api_key=api_key,
                        deployment_name=deployment_name,
                        endpoint = endpoint
    )
)

#### Definiendo nuestra "Función de Selección de Agente"

In [9]:
# Se define una función de prompt que decidirá qué agente debe hablar a continuación.
# ¡Esto es una IA decidiendo qué otra IA debe actuar!
selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""
Examina la RESPUESTA proporcionada y elige el siguiente participante.
Indica únicamente el nombre del participante elegido sin dar explicaciones.
Nunca elijas al participante nombrado en la RESPUESTA.

Elige solo entre estos participantes:
- {REVIEWER_NAME}
- {WRITER_NAME}

Reglas:
- Si la RESPUESTA es del usuario, es el turno de {WRITER_NAME}.
- Si la RESPUESTA es de {REVIEWER_NAME}, es el turno de {WRITER_NAME}.
- Si la RESPUESTA es de {WRITER_NAME}, es el turno de {REVIEWER_NAME}.

RESPUESTA:
{{{{$lastmessage}}}}
""",
)

#### Definiendo nuestra "Función de Terminación de Conversación"

In [10]:
# Se define una función de prompt que decidirá si la conversación ha terminado.
termination_keyword = "yes"

termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt=f"""
Examina la RESPUESTA y determina si el contenido se ha considerado satisfactorio.
El contenido se considera satisfactorio si el revisor ha dado una puntuación de 8 o superior y ha dicho "El artículo está listo para publicarse."
Si el contenido es satisfactorio, responde con una sola palabra sin dar explicaciones: {termination_keyword}.

RESPUESTA:
{{{{$lastmessage}}}}
""",
    )

#### Creando nuestro "Chat Grupal" de Agentes

In [11]:
# Reduce el historial de chat para no exceder el límite de tokens, manteniendo los 5 mensajes más recientes.
history_reducer = ChatHistoryTruncationReducer(target_count=5)

# Se crea la instancia del Chat Grupal, configurando las estrategias de control.
chat = AgentGroupChat(
        # Se especifica qué agentes participarán en el chat.
        agents=[agent_reviewer, agent_writer],
        # Se define la estrategia para decidir quién habla a continuación.
        selection_strategy=KernelFunctionSelectionStrategy(
            initial_agent=agent_writer, # Se define que el Escritor siempre empieza después del usuario.
            function=selection_function, # La función de prompt que tomará la decisión.
            kernel=kernel, # El kernel que ejecutará la función.
            result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else WRITER_NAME, # Procesa la respuesta para obtener solo el nombre.
            history_variable_name="lastmessage", # El nombre de la variable en el prompt que contendrá el último mensaje.
            history_reducer=history_reducer,
        ),
        # Se define la estrategia para decidir cuándo terminar el chat.
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[agent_reviewer], # Solo el Revisor puede terminar la conversación.
            function=termination_function, # La función de prompt que decidirá si se termina.
            kernel=kernel,
            result_parser=lambda result: termination_keyword in str(result.value[0]).lower(), # Procesa la respuesta para ver si contiene la palabra clave.
            history_variable_name="lastmessage",
            maximum_iterations=10, # Un límite de seguridad para evitar bucles infinitos.
            history_reducer=history_reducer,
        ),
    )

#### Ejecutando nuestro Chat Grupal de Agentes

In [13]:
# Mensaje de bienvenida para el usuario.
print(
        "¡Listo! Escribe tu artículo inicial, o 'exit' para salir, 'reset' para reiniciar la conversación."
    )

# Bucle principal para la interacción del usuario.
is_complete = False
while not is_complete:
        print()
        # Se obtiene el texto inicial del usuario.
        user_input = input("Usuario > ").strip()
        if not user_input:
            continue

        # Comandos para controlar el chat.
        if user_input.lower() == "exit":
            is_complete = True
            break
        if user_input.lower() == "reset":
            await chat.reset()
            print("[La conversación ha sido reiniciada]")
            continue
        
        # Se añade el mensaje del usuario al historial del chat para dar comienzo al proceso.
        await chat.add_chat_message(message=user_input)

        try:
            # ¡La magia ocurre aquí! 'chat.invoke()' inicia el ciclo de conversación automática.
            # El chat gestionará los turnos entre el Escritor y el Revisor hasta que la estrategia de terminación se cumpla.
            async for response in chat.invoke():
                if response is None or not response.name:
                    continue
                # Se imprime el mensaje de cada agente a medida que 'hablan'.
                print()
                print(f"# {response.name.upper()}:\n{response.content}")
        except Exception as e:
            print(f"Error durante la invocación del chat: {e}")

        # Se reinicia el indicador de finalización para la siguiente ronda de conversación.
        chat.is_complete = False

¡Listo! Escribe tu artículo inicial, o 'exit' para salir, 'reset' para reiniciar la conversación.


# ESCRITOR:
La inteligencia artificial está transformando nuestro mundo a gran velocidad. Observamos asistentes de IA en dispositivos móviles y hogares, facilitando tareas cotidianas. Las empresas también la incorporan para aumentar su eficiencia. El futuro de la IA se vislumbra prometedor y lleno de innovaciones que simplificarán nuestra vida. Esta revolución tecnológica avanza a pasos agigantados.

# REVISOR:
1. El artículo es claro, preciso e interesante, presentando la IA como una herramienta transformativa en la vida diaria y empresarial. Sin embargo, podría beneficiarse de ejemplos más concretos.
2. Puntuación: 8
3. Sugerencias para mejorar:
   - Incluir ejemplos específicos de tareas diarias o soluciones empresariales en las que se utiliza la IA.
   - Añadir un dato o una estadística para resaltar el impacto de la IA.
   - Considerar mencionar posibles desafíos o consideraciones é